In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sparse_dot_topn import awesome_cossim_topn 
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from deduping_module import deduping_class

In [2]:
link = r'C:\Users\USER\Documents\LM_project\SAL-230_Revisit validity'
gt = pd.read_csv(link + '\\raw\sf_export.csv', encoding='latin-1')
nm = pd.read_csv(link + '\\raw\pennsylvania-00001.csv', encoding='latin-1')

In [3]:
gt

,Salesforce Account Id,Salesforce Contact Id,Account Name,First Name,Last Name,Email,Phone,Zip Code 1,County 1,State 1,Quarantine
0,0014y00002LvzGCAAZ,0034y00002QqcZtAAJ,Melcher Law,Doug,Melcher,NaN,(617) 485-0859,NaN,NaN,NaN,0.0
1,0014y00002aVU9TAAW,0034y00002ZjWOiAAN,Kevin Barnett,Kevin,Barnett,kb@barnettfalls.com,NaN,NaN,NaN,NC,0.0
2,0014y00002aVjVkAAK,0034y00002ZjlorAAB,Kellie Jenkins Hohenshelt,Kellie Jenkins,Hohenshelt,NaN,(214) 587-7044,NaN,NaN,TX,0.0
3,0014y00002aVbeeAAC,0034y00002ZjdxQAAR,Jackalynne Fletcher,Jackalynne,Fletcher,NaN,(614) 487-8283,NaN,NaN,OH,0.0
4,0014y00002aVn5XAAS,0034y00002ZjpOtAAJ,Timothy Blair,Timothy,Blair,NaN,(713) 818-8474,NaN,NaN,TX,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1537649,All contacts and account,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1537650,"Copyright (c) 2000-2022 salesforce.com, inc. A...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1537651,Confidential Information - Do Not Distribute,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1537652,Generated By: Raffie Navaluna 8/30/2022 9:22 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
gt.dropna(subset= ['Salesforce Contact Id'], inplace= True)
gt.drop_duplicates(subset=['Salesforce Account Id'], inplace= True) # need to remove the duplicates in the salesforce account id if we dedupe on the account object
gt.reset_index(drop=True, inplace= True) #always make sure that the index of the inputs are in numerical order or this will cause errors in getting the matches
gt.info()

Note:
1. the footer on the ground truth file should always be dropped

In [ ]:
# initializing the module
deduping = deduping_class(gt) 

In [ ]:
deduping.ground_truth.head()

#### Step 1:
1. Make sure that the ground truth columns are similar with the dataframe to be matched

In [ ]:
# matching the ground truth columns with the to match dataframe
nm.rename(columns={'firstName':'First Name', 'lastName':'Last Name', 'email':'Email', 'phone':'Phone', 'zip':'Zip Code 1'}, inplace= True)

In [ ]:
nm.head(3)

#### Step 2
Now that the columns are matched, we can use the key_selector function. \
This takes 2 inputs, *args and "data=dataframe to be matched"\
This function will return the to be matched dataframe with primary_key value while also adding it in the ground truth but only in the backend

In [ ]:
deduping.key_selector('First Name', 'Last Name', 'Phone', data= nm)

In [ ]:
deduping.nm['state'] = deduping.nm.apply(lambda x: deduping.state_abbrev(x['state']), axis= 1)

#### Optional Step
We can set an optional paramater called ngrams, this means the number of combination the txt will be divided. \
if this is not set, it will automatically equal to 3

In [ ]:
deduping.set_ngrams(4)

#### Step 3
This function will convert the ground truth and dataframe to be matched in to tfidf sparse matrix \
the input to this function should be the dataframe with the primarykey \
the results will now be included in the class module

self.nm_tfidf\
self.gt_tfidf

In [ ]:
deduping.vectorizer()

#### Step 4
This function will get compare the similarities in the 2 dataframe and save the output in self.match

In [ ]:
deduping.get_match(2000)

In [ ]:
deduping.matched

#### Notes:
1. If deduping at account object, always remember to remove the duplicates in the Salesforce Account Id
1. always make sure that the index of the inputs are in numerical order or this will cause errors in getting the matches
1. if a selected key value is missing, it has a significant impact on the performance